In [2]:
# USAGE
# python detect_mask_image.py --image examples/example_01.png

# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os
from CentroidBasedObjectDetector import CentroidTracker
from collections import deque
from collections import Counter
import imutils
import _thread as thread
import uuid

In [3]:
maxQueueLen = 20
developer_mode = True

In [4]:
args = {}

if developer_mode == False:
# construct the argument parser and parse the arguments
    ap = argparse.ArgumentParser()
    ap.add_argument("-i", "--image", required=True,help="path to input image"),
    ap.add_argument("-c", "--capture_type",default = 0,required=True,help="0 for standard camera, 1 for RTSP stream,2 for video file.")
    ap.add_argument("-r", "--source-url", type=str ,help="The url of the RTSP camera stream associated with this instance, or the video file to play.")
    ap.add_argument("-r", "--device-index", type=int,help="Standard camera device index.")
    ap.add_argument("-d", "--detection_model", required=True,help="path to Face detection model.")
    ap.add_argument("-p", "--prototxt", required=True,help="path to Caffe 'deploy' prototxt file.")
    ap.add_argument("-m", "--model", type=str,default="mask_detector.model",help="path to trained face mask detector model.")
    ap.add_argument("-c", "--confidence", type=float, default=0.5,help="minimum probability to filter weak detections.")
    ap.add_argument("-d", "--interface-page", type=str,help="The interface page path.",required=True)
    ap.add_argument("-t", "--interface-template", type=str,help="The interface template page path.",required=True)
    ap.add_argument("-n", "--image-dir", type=str,help="The image directory of the application..",required=True)
    ap.add_argument("-s", "--skip-frames", type=int, default=30,help="# of skip frames between detections")
    
    args = vars(ap.parse_args())
else:
    args = {"image":r"D:\DataScience\Notebooks\ComputerVision\images\capture.png",
           "prototxt":"Models/deploy.prototxt",
            "detection_model": "Models/fd_ov_m.caffemodel",
            "model":"models/mask_detector.model",
            "confidence":0.5,
            "capture_type" : 2,
            "source-url": "mask.mp4",
            "device-index" : 0,
            "skip_frames" : 0,
            "interface_page":r"C:\Users\Ahmed Omar\Source\Repos\DTCM_Platform\DTCM_Platform\MaskDetectorPage\index.html",
            "interface_template":r"C:\Users\Ahmed Omar\Source\Repos\DTCM_Platform\DTCM_Platform\MaskDetectorPage\index_Template.html",
            "image_dir":r"C:\Users\Ahmed Omar\Source\Repos\DTCM_Platform\DTCM_Platform\MaskDetectorPage\Images"
           }
ct = CentroidTracker()

In [5]:

# load our serialized face detector model from disk
#print("[INFO] loading face detector model...")
prototxtPath = args["prototxt"]
weightsPath = args["detection_model"]
net = cv2.dnn.readNet(prototxtPath, weightsPath)


# load the face mask detector model from disk
#print("[INFO] loading face mask detector model...")
model = load_model(args["model"])

skip_frames = args["skip_frames"]


In [5]:
interface_template = ""

with open(args["interface_template"],"r") as file:
    interface_template = file.read()

interface_data = interface_template

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Ahmed Omar\\Source\\Repos\\DTCM_Platform\\DTCM_Platform\\MaskDetectorPage\\index_Template.html'

In [ ]:
filelist = [ f for f in os.listdir(args["image_dir"]) if f.endswith(".png") ]
for f in filelist:
    os.remove(os.path.join(args["image_dir"], f))

In [6]:
def UpdateWebPageStatus(result):
    global interface_template
    global interface_data
    interface_data = interface_template
    while True:
        try:
            no_mask_count = 0
            images = ""
            for r in result:
                if r["status"] == "Mask":
                    continue
                no_mask_count += 1
                face = r["img"]
                img_name = str(uuid.uuid4())+ ".png"
                face = imutils.resize(face,height = 150)
                cv2.imwrite(os.path.sep.join([args["image_dir"], img_name]),face)
                images += "<td><img src=" +"Images/" +  img_name + " /></td>"
                    
            interface_data = interface_data.replace("{count}",str(no_mask_count))
            interface_data = interface_data.replace("{ImagePlaceHolder}",images)
            #print(interface_data)
            while True:
                count2 = 0
                try:
                    with open(args["interface_page"], "w") as f:
                        f.write(interface_data)     
                       # print(interface_data)
                        f.close()
                        break
                except Exception as ex2:
                    count2 +=1
                    if count2>=100:
                        break
                    print(ex2)
                    continue
            break
        except Exception as ex:
            count +=1
            if count>=100:
                break
            print(ex)
            continue

In [ ]:


# UpdateWebPageStatus(result)

In [7]:
def process_image(image):
    result = []
    orig = image.copy()
    (h, w) = image.shape[:2]
    rects = []
    # construct a blob from the image
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),
    (104.0, 177.0, 123.0))


    # pass the blob through the network and obtain the face detections

    net.setInput(blob)
    detections = net.forward()

    for i in range(0, detections.shape[2]):
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        confidence = detections[0, 0, i, 2]

        if confidence > args["confidence"]:
            rects.append((startX,startY,endX,endY))
        
    objects = ct.update(rects)
 
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the detection
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the confidence is
        # greater than the minimum confidence
        if confidence > args["confidence"]:
    
            # compute the (x, y)-coordinates of the bounding box for
            # the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # ensure the bounding boxes fall within the dimensions of
            # the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extract the face ROI, convert it from BGR to RGB channel
            # ordering, resize it to 224x224, and preprocess it
            face = image[startY:endY, startX:endX]
            if face is None or len(face) == 0:
                continue
            face_crop = face.copy()
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
    #         cv2.imshow("Image",face)
    #         cv2.waitKey(0)
    #         cv2.destroyAllWindows()
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)

            # pass the face through the model to determine if the face
            # has a mask or not
            (mask, withoutMask) = model.predict(face)[0]

            # determine the class label and color we'll use to draw
            # the bounding box and text
            label = "Mask" if mask > withoutMask else "No Mask"


            # include the probability in the label
          #  label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

            # display the label and bounding box rectangle on the output
            # frame
            
            rolling_ix = list(objects.keys())[i]
            if rolling_ix in Rolling_Masks.keys():
                Rolling_Masks[rolling_ix].append(label)
            else:
                Q_g = deque(maxlen = maxQueueLen)
                Q_g.append(label)
                Rolling_Masks[rolling_ix] = Q_g
            
            status = Counter(Rolling_Masks[rolling_ix]).most_common(1)[0][0]
            #print(str(rolling_ix) + status)
            #print("===========")
            result.append({"index":rolling_ix,"status":status, "img":face_crop })
            color = (0, 255, 0) if status == "Mask" else (0, 0, 255)
            mask_status = "Warning: NOT WEARING A PROTECTIVE MASK"
            if status == "Mask":
                mask_status = "Passed: Weraing a protective mask."
            cv2.putText(image, mask_status, (startX, startY - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(image, (startX, startY), (endX, endY), color, 4)

    # show the output image

    return image,result

In [ ]:
# img = cv2.imread(r"C:\Users\Ahmed Omar\Desktop\img.jpg")
# Rolling_Masks = {}
# current_skip = 0
# img,_ =process_image(img)

# cv2.imshow("img",img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [9]:
video = cv2.VideoCapture(args["source-url"])
Rolling_Masks = {}
current_skip = 0

skip_frames =1

while True:
    ret,frame = video.read()
    if current_skip<=skip_frames:       
        current_skip += 1
        continue
    else:
        current_skip = 0

        
    number_nomask = 0
    faces_nomask = []
 
    if frame is None or ret == False:
        break
    frame,result = process_image(frame)
    cv2.imshow("Output", frame)
#     for d in result:
#         if d["status"] == "No Mask":
#             number_nomask += 1
#             faces_nomask.append(d["img"])
#             #cv2.imshow(str(d["index"]),d["img"])
            
  #  thread.start_new_thread(UpdateWebPageStatus,(result,))
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    
    # Pefrom external Processing here
    
cv2.destroyAllWindows()
video.release()